# Checking ML binary classification scores

## Brief introduction

There are numerous performance measures (scores) of binary classification (see the colorful table at https://en.wikipedia.org/wiki/F-score) that authors report regularly, usually following incorrect evaluation protocols, adding cosmetics, typos, etc.

Nevertheless, these scores provide the basis for ranking the value of research in machine learning (and many related fields), influence the acceptance of papers and eventually skew entire fields (Attila can talk about our discoveries in retinal vessel segmentation).

Generally, all these performance measures are calculated in the following manner. The proposed ML technique is evaluated on test set (independent from the one it is trained on), predictions of binary labels are made, and four raw figures (the confusion matrix is determined):
* tp (the number of test instances that were labelled positive in the test set and got predicted positive by ML);
* tn (the number of test instances that were labelled negative in the test set and got predicted negative by ML);
* fp (the number of test instances that were labelled negative in the test set but got predicted positive by ML);
* fn (the number of test instances that were labelled positive in the test set but got predicted negative by ML).

Denoting the total number of positive and negative test items by p and n respectively, it can be readily seen that p = tp + fn and n = tn + fp.

The goal of the various performance measures is to unify these four figures into one single score that can be used to rank various ML approaches. The reason for the numerous types of scores it that they reflect various aspects of the relation of these four figures.

However, we usually know the properties of the test set, that is, we know p and n. Then, since p = tp + fn and n = tn + fp, we can state that all performance scores depend on two variables only. Without the loss of generality, we pick tp and tn.

## Example

The idea of our research is to check the internal consistency of performance scores published by authors to recognize the corruption of the research. If authors publish at least three performance scores, for example, 

acc = (tp + tn)/(p + n)

sens = tp/p

spec = tn/n

then utilizing that we know p and n (from the publicly available dataset), we can pick two scores (for example, sensitivity and specificity) and reconstruct

tp* = sens * p

tn* = spec * n

Then, we can calculate

acc* = (tp* + tn*)/(p + n)

and we can check how acc* relates to acc. If all scores are consistent, they should be close to each other (supposing that the authors only applied rounding to k digits, we can exactly tell the true interval of acc* and check if the published acc score is in that). If the scores fail this check of internal consistency, it means that the authors either applied cosmetics, or adjusted the dataset (altering p and n) for example by removing those instances which are hard to label by ML.

This kind of cheating is incredibly common and harmful to science.

## The algebraic problem and the ultimate goal

The example above is using the simplest scores. The ultimate goal is to provide a list of formulas/solutions for any pair of performance scores to reconstruct tp and tn (always two equations dependent on tp and tn, thus, we should be able to solve them). Then, given any pair of scores, being able to reconstruct tp and tn, we can estimate any other score and compare it against the published value.

Many of the scores include higher-order terms in tp and tn, which makes the solution a little bit more complicated. In the example below I use computer algebra (sympy) to derive some formulas for two more complex scores and point out the main problems.

Suppose there is a work, where authors published the diagnostic odds ratio (dor), the Fowlkes-Mallows index (fm) and the accuracy and we know the dataset (p=100, n=200):

acc = 0.943 # (tp + tn)/(p + n)

dor = 0.345 # (tn * tp) / ((n - tn) * (p - tp))

fm = 0.543 # tp/sqrt(p*(n - tn + tp))

We pick dor and fm to reconstruct tp and tn, and then we want to calculate acc* and compare it to the published acc score to check the internal consistency. dor and fm are two equations in two independent variables (tp and tn) and ideally, we would like to drive formulas like

tp* = formula_0(dor, fm, p, n)

tn* = formula_1(dor, fm, p, n)

In [1]:
import sympy as sp

n = sp.Symbol('n')
p = sp.Symbol('p')
tp = sp.Symbol('tp')
tn = sp.Symbol('tn')
dor = sp.Symbol('dor')
fm = sp.Symbol('fm')

eq_dor = (tn * tp) / ((n - tn) * (p - tp)) - dor
eq_fm = tp/sp.sqrt(p * (n - tn + tp)) - fm

## The polynomial equation system to solve

Supposing that the denominators are not 0, we can multiply by them and then the equations are really in polynomial format.

In [2]:
eq_dor

-dor + tn*tp/((n - tn)*(p - tp))

In [3]:
eq_fm

-fm + tp/sqrt(p*(n - tn + tp))

## Solving the equation system

In [4]:
# Lets solve the dor equation for tp
sol_dor = sp.solve(eq_dor, tp)

In [5]:
# There is one solution (lucky case, there could be two if tp was squared)
sol_dor

[dor*p*(n - tn)/(dor*n - dor*tn + tn)]

In [6]:
# Let's substitute this into the fm equation
eq_fm_subs = eq_fm.subs({tp: sol_dor[0]})

In [7]:
# The equation to solve for tn
eq_fm_subs

dor*p*(n - tn)/(sqrt(p*(dor*p*(n - tn)/(dor*n - dor*tn + tn) + n - tn))*(dor*n - dor*tn + tn)) - fm

In [8]:
# solving for tn
sol_fm = sp.solve(eq_fm_subs, tn)

In [9]:
# there are two solutions
sol_fm

[dor*(2*dor*fm**2*n + dor*fm**2*p - dor*p - 2*fm**2*n - fm**2*p - sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))/(2*fm**2*(dor**2 - 2*dor + 1)),
 dor*(2*dor*fm**2*n + dor*fm**2*p - dor*p - 2*fm**2*n - fm**2*p + sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))/(2*fm**2*(dor**2 - 2*dor + 1))]

In [10]:
tn_0, tn_1 = sol_fm

In [11]:
# lets substitute the solutions for tn into tp
tp_0 = sp.simplify(sol_dor[0].subs({tn: tn_0}))
tp_1 = sp.simplify(sol_dor[0].subs({tn: tn_1}))

## The two pairs of solutions

In [12]:
# tp_0
tp_0

p*(dor**2*fm**2*p - dor**2*p + 2*dor*fm**2*n - dor*fm**2*p - dor*sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)) - 2*fm**2*n)/(dor**2*fm**2*p - dor**2*p - 2*dor*fm**2*p + dor*p - dor*sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)) + fm**2*p + sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))

In [13]:
# tn_0
tn_0

dor*(2*dor*fm**2*n + dor*fm**2*p - dor*p - 2*fm**2*n - fm**2*p - sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))/(2*fm**2*(dor**2 - 2*dor + 1))

In [14]:
# tp_1
tp_1

p*(dor**2*fm**2*p - dor**2*p + 2*dor*fm**2*n - dor*fm**2*p + dor*sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)) - 2*fm**2*n)/(dor**2*fm**2*p - dor**2*p - 2*dor*fm**2*p + dor*p + dor*sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)) + fm**2*p - sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))

In [15]:
# tn_1
tn_1

dor*(2*dor*fm**2*n + dor*fm**2*p - dor*p - 2*fm**2*n - fm**2*p + sqrt(p*(dor**2*fm**4*p - 2*dor**2*fm**2*p + dor**2*p - 2*dor*fm**4*p - 4*dor*fm**2*n + 2*dor*fm**2*p + fm**4*p + 4*fm**2*n)))/(2*fm**2*(dor**2 - 2*dor + 1))

## The check

Now we can estimate tp and tn in two different ways, by substituting the dor and fm values into any pair of the solutions

In [16]:
subs = {fm: 0.543,
        dor: 0.345,
        p: 100,
        n: 200}

In [17]:
# the first pair

tp_0_prime = tp_0.subs(subs)
tn_0_prime = tn_0.subs(subs)

acc_0_prime = (tp_0_prime + tn_0_prime)/(100 + 200)

In [18]:
tp_0_prime, tn_0_prime, acc_0_prime

(-114.988069506881, -363.429682974090, -1.59472584160324)

In [19]:
# the second pair

tp_1_prime = tp_1.subs(subs)
tn_1_prime = tn_1.subs(subs)

acc_1_prime = (tp_1_prime + tn_1_prime)/(100 + 200)

In [20]:
tp_1_prime, tn_1_prime, acc_1_prime

(91.8012137816902, 5.97820431611338, 0.325931393659345)

Regarding the reconstructed tp, tn and acc scores with the first pair of solutions, it can be seen that it is not physical, tp, tn and acc cannot be negative.

The second formula seems to be the correct choice in this case, which gives a physical value for tp (91) and tn (6), however, accuracy is very much off from the published score (0.943) which shows that the results are inconsistent.

IMPORTANT NOTE: So far we were vague with when do we call the reconstructed score to be 'close' to the published one. This is because in this description I focus on the formulas we need. In practice, we treat the published scores as intervals. For example, if a score 0.543 is published for FM, then we can suppose that the score is rounded, maybe floored or ceiled to the third digit, which means that the original score fad to fall in the interval [0.542, 0.544] with certainty. The same is true for the published 0.345 for DOR, it had to fall into the interval [0.344, 0.346].

If we had the solution formulas (that we just derived) for tp and tn, we can substitute these intervals into the formulas and also into the calculation of acc*. Then, by the rules of interval computing, the result for acc* will be another interval and if the scores are consistent, the interval for the published acc score [0.942, 0.944] and the interval reconstructed for acc* needs to have an intersection. If they are disjoint, then the published scores are inconsistent with certainty under the assumption of ceiling/rounding to the third digits and assuming p and n being the statistics of the dataset.

This is a hard decision on the consistency of the scores, which gives a tool to recognize certain types of "cheating". Naturally, this technique cannot recognize all issues. However, if this test of consistency fails, it is undoubtable that the scores are not calculated by the preferred protocol.

## The challenges

First of all, there are numerous pairs of scores and we want to have the tp and tn solution formulas for each pair to provide a general tool for researchers to check the consistency of scores. Potentially, substituting some given p and n values would make the symbolic solving easier a bit (the solver would see that they are positives), but in that case each check would require the involvement of symbolic toolkits like sympy which would make things unnecessarily complicated. Thus, the goal is to have tp and tn formulas (with the symbols p and n) for each potential pair of performance scores.

The real challenges come with the interpretation of the results:
* Already this example shows that there are two pairs of solutions for tp and tn. Depending on the complexity of the scores, there can be as many as 8 different pairs of solutions. What is the correct interpretation of these many formulas? Can we always expect one of them to give physical values? By 'physical' I mean 0 <= tp <= p and 0 <= tn <= n. What is the right interpretation in terms of consistency if none of them or all of them give physical values? (I guess in the first case the scores are not consistent, in the second case the union of the results must be taken).
* Slightly more complicated questions are what happens if the formulas don't work? Already in this example, one can see that there are denominators and square roots as well. With more complicated scores I have seen fractions under square roots, and square roots under square roots as well. What does it mean regarding the concistency if a denominator in one of the solutions becomes 0? What does it mean if some denominator in all solutions become 0? My guess is that this indicates an underdetermined system (at least this is what I observed with the simple cases). However, what does it mean if the value under the square root becomes negative in one solution (remember there can be even 8 different pairs of solutions), or what does it mean if some value under some square root becomes negative in all solutions?

With my limited understanding of algebra, I started to experiment with Gröbner bases. Given a system of two such polynomial equations, I was able to determine their Gröbner basis automatedly and I learnt that the system is solvable if the coefficients of the leading monomials in the Gröbner basis fulfil some requirements. I tended to recognize that these coefficients might appear as denominators. Hence, the disappearance of these coefficients might lead to unsolvable systems, and I kind of concluded that this is the reason why if a denominator becomes zero, then essentially the coefficient of a leading monomial in the Gröbner basis became zero, therefore, the system cannot be solved, it is underdetermined.

However, this is rather an opinion according to my best understanding (based on some observations) than something sound enough to rely our package on. Anyways, I shared it in the hope it might be useful.

I have no idea how to interpret negatives under square roots.

If we could develop some systematic/algorithmic approach to these interpretations, like (sorry for this nonsense, only an illustration)
* Step 1: determine the Gröbner basis of the system;
* Step 2: collect all coefficients of leading monomials;
* Step 3: solve each of them for zero - in all these cases the problem is not solvable;
* Step 4: check for square roots - solve all expression under the roots for negatives. In all these cases the system is not solvable,

then I could easily implement it to have a general solution.

